In [2]:
import numpy as np
import sys

sys.path.append("/home/chase/applications/pydec/")

import pydec
from scipy.sparse import linalg as sp

#flows
X_simple = np.array([[0, 5, 0, 0], [-5, 0, 3, -1], [0, -3, 0, 1], [0, 1, -1, 0]])
X_bad = np.array([[0, 1, 0, 0, 0], [-1, 0, 2, -1, 0], [0, -2, 0, 1, 1], [0, 1, -1, 0, 0], [0, 0, -1, 0, 0]])
X_good = np.array([[0, 3, 0, 0, 0], [-3, 0, 2, 1, 0], [0, -2, 0, -1, 3], [0, -1, 1, 0, 0], [0, 0, -3, 0, 0]])
#adjacency
A = X_simple.copy()
A_gb = X_bad.copy()
A[A != 0] = 1
A_gb[A_gb != 0] =1

#Laplacian
L = np.sum(X_simple, 1)*np.eye(4,4) - A
L_good = np.sum(X_good, 1)*np.eye(5,5) - A_gb
L_bad = np.sum(X_bad, 1)*np.eye(5,5) - A_gb

In [3]:
#data = np.loadtxt('/home/chase/projects/hodge-grid/src/python/flow_example.txt')
data_good = np.loadtxt('/home/chase/projects/hodge-grid/src/python/flow_example_good.txt')
data_bad = np.loadtxt('/home/chase/projects/hodge-grid/src/python/flow_example_bad.txt')


In [4]:
edges_good = data_good[:,:2]
edges_bad = data_bad[:,:2]

#create simplicial complex
asc = pydec.abstract_simplicial_complex([edges_good, [1,2,3]]) #manually add triangle
print(asc.simplices)

[array([[ 0.],
       [ 1.],
       [ 2.],
       [ 3.],
       [ 4.]]), array([[ 0.,  1.],
       [ 1.,  2.],
       [ 1.,  3.],
       [ 2.,  3.],
       [ 2.,  4.]]), array([[1, 2, 3]])]


/home/chase/applications/pydec/pydec/dec/simplex_array.py:28: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if rank(s) != 2 or rank(v) != 2:


In [10]:
omega = data_good[:,-1] #flow data
cmplx = asc.chain_complex()
b1 = cmplx[1].astype(float) #boundary matrix
b2 = cmplx[2].astype(float)

print(b1.shape)
print("\n")
print(b2)
print(b2.shape)

(5, 5)


  (1, 0)	1.0
  (2, 0)	-1.0
  (3, 0)	1.0
(5, 1)


In [29]:
x = np.ones((b1.shape[1],1))

alpha = sp.cg(b1.dot(b1.T), b1.dot(x), tol=1e-8)[0]
beta = sp.cg(b2.T.dot(b2), b2.T.dot(x), tol=1e-8)[0]


In [30]:
print(b1.shape)
print(alpha.shape)
print(b2.shape)
print(beta.shape)

(5, 5)
(5,)
(5, 1)
(1,)


In [34]:
h = x - (b1.T.dot(alpha)) - (b2.dot(beta))
print(h.shape)

(5, 5)


In [24]:
print(b1)

  (0, 0)	-1.0
  (1, 0)	1.0
  (1, 1)	-1.0
  (1, 2)	-1.0
  (2, 1)	1.0
  (2, 3)	-1.0
  (2, 4)	-1.0
  (3, 2)	1.0
  (3, 3)	1.0
  (4, 4)	1.0


In [25]:
print(b2)

  (1, 0)	1.0
  (2, 0)	-1.0
  (3, 0)	1.0


In [35]:
L_good.dot(h) #harmonic solution, should equal 0 when mulitplied by Laplacian

array([[  2.22044605e-16,  -5.55111512e-16,  -3.33066907e-16,
          1.11022302e-16,   2.22044605e-16],
       [ -3.33066907e-16,   8.32667268e-16,   4.99600361e-16,
         -1.66533454e-16,  -3.33066907e-16],
       [ -3.33066907e-16,   8.32667268e-16,   4.99600361e-16,
         -1.66533454e-16,  -3.33066907e-16],
       [ -2.22044605e-16,   5.55111512e-16,   3.33066907e-16,
         -1.11022302e-16,  -2.22044605e-16],
       [ -4.44089210e-16,   1.11022302e-15,   6.66133815e-16,
         -2.22044605e-16,  -4.44089210e-16]])

In [10]:
X_good*alpha #gradient

array([[-0.        , -1.6       ,  0.        ,  0.        ,  0.        ],
       [ 4.6       , -0.        ,  0.26666667,  0.8       ,  0.        ],
       [-0.        ,  1.06666667,  0.        , -0.8       ,  3.4       ],
       [-0.        ,  0.53333333,  0.13333333,  0.        ,  0.        ],
       [-0.        , -0.        , -0.4       ,  0.        ,  0.        ]])

In [11]:
X_good*beta #curl free

array([[ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ],
       [-1.        ,  0.        ,  0.66666667,  0.33333333,  0.        ],
       [ 0.        , -0.66666667,  0.        , -0.33333333,  1.        ],
       [ 0.        , -0.33333333,  0.33333333,  0.        ,  0.        ],
       [ 0.        ,  0.        , -1.        ,  0.        ,  0.        ]])

In [26]:
alpha

array([-1.53333333, -0.53333333,  0.13333333,  0.8       ,  1.13333333])

In [27]:
beta

array([ 0.33333333])

In [28]:
omega = data_bad[:,-1] #flow data
cmplx = asc.chain_complex()
b1 = cmplx[1].astype(float) #boundary matrix
b2 = cmplx[2].astype(float)

x = np.ones(b1.shape[1])

alpha = sp.cg(b1 * b1.T, b1*x, tol=1e-8)[0]
beta = sp.cg(b2.T * b2, b2.T * x, tol=1e-8)[0]

h = x - (b1.T *alpha) - (b2*beta)

In [32]:
h

array([  0.00000000e+00,  -1.66533454e-16,  -1.66533454e-16,
        -5.55111512e-17,   1.11022302e-16])

In [30]:
X_bad*alpha #gradient

array([[-0.        , -0.53333333,  0.        ,  0.        ,  0.        ],
       [ 1.53333333, -0.        ,  0.26666667, -0.8       ,  0.        ],
       [-0.        ,  1.06666667,  0.        ,  0.8       ,  1.13333333],
       [-0.        , -0.53333333, -0.13333333,  0.        ,  0.        ],
       [-0.        , -0.        , -0.13333333,  0.        ,  0.        ]])

In [31]:
X_bad*beta #curl free

array([[ 0.        ,  0.33333333,  0.        ,  0.        ,  0.        ],
       [-0.33333333,  0.        ,  0.66666667, -0.33333333,  0.        ],
       [ 0.        , -0.66666667,  0.        ,  0.33333333,  0.33333333],
       [ 0.        ,  0.33333333, -0.33333333,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.33333333,  0.        ,  0.        ]])

In [35]:
alpha

array([-1.53333333, -0.53333333,  0.13333333,  0.8       ,  1.13333333])

In [36]:
beta

array([ 0.33333333])

In [37]:
np.linalg.pinv(L_good)

array([[ 0.38888889,  0.16666667, -0.16666667, -0.22222222,  0.05555556],
       [ 0.16666667,  0.5       , -0.5       , -0.66666667,  0.16666667],
       [-0.16666667, -0.5       ,  0.5       , -0.33333333, -0.16666667],
       [-0.22222222, -0.66666667, -0.33333333,  0.55555556,  0.11111111],
       [ 0.05555556,  0.16666667, -0.16666667,  0.11111111, -0.27777778]])

In [38]:
np.linalg.pinv(L_bad)

array([[  1.50000000e+00,   5.00000000e-01,  -5.00000000e-01,
         -1.00000000e+00,   5.00000000e-01],
       [  5.00000000e-01,   5.00000000e-01,  -5.00000000e-01,
         -1.00000000e+00,   5.00000000e-01],
       [ -5.00000000e-01,  -5.00000000e-01,   5.00000000e-01,
          0.00000000e+00,  -5.00000000e-01],
       [ -1.00000000e+00,  -1.00000000e+00,   2.22044605e-16,
          1.00000000e+00,  -2.77555756e-16],
       [  5.00000000e-01,   5.00000000e-01,  -5.00000000e-01,
          1.11022302e-16,  -5.00000000e-01]])

In [49]:
#rips complex example
pts = pydec.read_array('/home/chase/projects/opf/src/python/pydec/Examples/RipsComplex/300pts.mtx')
rc = pydec.rips_complex(pts, 0.15)
print(rc.simplices)

[array([[  0],
       [  1],
       [  2],
       [  3],
       [  4],
       [  5],
       [  6],
       [  7],
       [  8],
       [  9],
       [ 10],
       [ 11],
       [ 12],
       [ 13],
       [ 14],
       [ 15],
       [ 16],
       [ 17],
       [ 18],
       [ 19],
       [ 20],
       [ 21],
       [ 22],
       [ 23],
       [ 24],
       [ 25],
       [ 26],
       [ 27],
       [ 28],
       [ 29],
       [ 30],
       [ 31],
       [ 32],
       [ 33],
       [ 34],
       [ 35],
       [ 36],
       [ 37],
       [ 38],
       [ 39],
       [ 40],
       [ 41],
       [ 42],
       [ 43],
       [ 44],
       [ 45],
       [ 46],
       [ 47],
       [ 48],
       [ 49],
       [ 50],
       [ 51],
       [ 52],
       [ 53],
       [ 54],
       [ 55],
       [ 56],
       [ 57],
       [ 58],
       [ 59],
       [ 60],
       [ 61],
       [ 62],
       [ 63],
       [ 64],
       [ 65],
       [ 66],
       [ 67],
       [ 68],
       [ 69],
       [ 70],
     

In [31]:
cmplx = rc.chain_complex()
b1 = cmplx[1].astype(float)
b2 = cmplx[2].astype(float)

x = np.random.rand(b1.shape[1])

alpha = sp.cg(b1 * b1.T, b1*x, tol=1e-8)[0]
beta = sp.cg(b2.T * b2, b2.T * x, tol=1e-8)[0]

h = x - (b1.T *alpha) - (b2*beta)
h /= abs(h).max()

In [32]:
print(h)

[ 0.02495788 -0.00580069 -0.00410362 ...,  0.39048615 -0.29396785
  0.41946563]
